In [1]:
from itertools import combinations, permutations
import os
from random import choice
import re

from rhfe_tools.arecibo.fit_funcs import *
from rhfe_tools.arecibo.ao_lunar_plot import *
from rhfe_tools.fit import fit_line, plot_mesh, correlation_matrix, Fit, numeric_columns
from rhfe_tools.project import ortho_transform, LUNAR_RADIUS
from scipy.stats import binned_statistic_2d
import django
import fs.path
import numpy as np
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from fs.osfs import OSFS

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import *
# from mastspec.views import *
# from mastspec.forms import *
from plotter.spectrum_ops import *
from plotter_utils import modeldict, model_metadata_df


ModuleNotFoundError: No module named 'sympy'

In [ ]:
filter_df_from_queryset??

In [ ]:
metadata_df = model_metadata_df(ZSpec, ["observation"])
data_df = filter_df_from_queryset(ZSpec.objects.all(), r_star=True, scale_to='L0B_R0B')
zdf = pd.concat([metadata_df, data_df], axis=1)

In [ ]:
choice(ZSpec.objects.all()).filter_values()

In [ ]:
data_df

In [ ]:
ZSpec.virtual_filter_mapping

In [ ]:
for feature in zdf['feature'].unique():
    zdf[feature] = 0
    zdf.loc[zdf['feature'] == feature, feature] = 1

In [ ]:
keep_columns = [
    "incidence_angle","emission_angle","phase_angle",
    *zdf.feature.unique()
    ]

In [ ]:
zfilts = [f for f in ZSpec.filters]
zcomb = list(combinations(zfilts, 2))
zper3 = list(permutations(zfilts, 3))

In [ ]:
ratframe = []
for f1, f2 in zcomb:
    series = ratio(zdf, ZSpec, f1, f2)[0]
    series.name = 'r ' + f1 + ' ' + f2
    ratframe.append(series)
ratframe = pd.concat(ratframe, axis=1)
full_ratframe = pd.concat([
    zdf[keep_columns], ratframe
], axis=1)

In [ ]:
bandframe = []
for f1, f2, f3 in zper3:
    try:
        series = band_depth_custom(zdf, ZSpec, f1, f2, f3)[0]
    except ValueError: # usually bd outside range
        continue
    series.name = 'bd ' + f1 + ' ' + f3 + ' ' + f2
    bandframe.append(series)
bandframe = pd.concat(bandframe, axis=1)
full_bandframe = pd.concat([
    zdf[keep_columns], bandframe
], axis=1)

In [ ]:
slopeframe = []
for f1, f2 in zcomb:
    try:
        series = slope(zdf, ZSpec, f1, f2)[0]
    except ValueError: # usually bd outside range
        continue
    series.name = 'sl ' + f1 + ' ' + f2
    slopeframe.append(series)
slopeframe = pd.concat(slopeframe, axis=1)
full_slopeframe = pd.concat([
    zdf[keep_columns], slopeframe
], axis=1)

In [ ]:
correlation_matrix??

In [ ]:
ratio_corrs = correlation_matrix(full_ratframe)
ratio_corr_feats = ratio_corrs[zdf.feature.unique()].drop(zdf.feature.unique(), axis=0)
bd_corrs = correlation_matrix(full_bandframe)
bd_corr_feats = bd_corrs[zdf.feature.unique()].drop(zdf.feature.unique(), axis=0)
slope_corrs = correlation_matrix(full_slopeframe)
slope_corr_feats = slope_corrs[zdf.feature.unique()].drop(zdf.feature.unique(), axis=0)

In [ ]:
pd.concat([
    ratio_corr_feats,
    bd_corr_feats.drop(['incidence_angle', 'emission_angle', 'phase_angle'], axis=0),
    slope_corr_feats.drop(['incidence_angle', 'emission_angle', 'phase_angle'], axis=0)
]).to_csv('feature_correlation_matrix_2021_04_08_scaled_l1_r1.csv')

In [ ]:
[
    spec.filter_values() for spec in
    mspec.filter(observation__seq_id__iexact='mcam00126')
]

In [ ]:
m = choice(MSpec.objects.all())

In [ ]:
n.model

In [ ]:
search_list = [{'field': 'lon', 'term': [], 'begin': 137.35, 'end': 137.4},
 {'field': 'member', 'term': ['Dillinger']}]

In [ ]:
print(pretty_print_search_params(search_list))

In [ ]:
MSpec.objects.get(id__iexact=2338).filter_values(average_filters=True, scale_to=('l2','r2'))

In [ ]:
filter_df = filter_df_from_queryset(MSpec.objects.all(), average_filters=True, scale_to=('l2','r2'))
band_depth_min(filter_df, MSpec, 'l1_r1', 'l6_r6')

In [ ]:
from plotter_utils import filter_nan_rows, qlist

In [ ]:
%time qlist(MSpec.objects.all(), 'id')

In [ ]:
%%time
filter_nan_rows(
    MSpec.objects.all(),
    filter_df,
    ['l3', 'r5']
)

In [ ]:
queryset = MSpec.objects.all()
%time [i[0] for i in queryset.values_list('id')]

In [ ]:
filt_args = ['l3', 'r5']
queryset = MSpec.objects.all()
%time relevant_filters = filter_df[filt_args]
%time queryset_ids = [spec.id for spec in queryset]
%time queryset_relevant_filters = relevant_filters.loc[queryset_ids]
%time queryset.model.objects.filter(id__in=queryset_relevant_filters.dropna().index)

In [ ]:
from plotter.graph import truncate_queryset_for_missing_properties

In [ ]:
truncate_queryset_for_missing_properties?

In [ ]:
settings = {
    'main-filter-1-marker.value': 'l2', 
    'main-filter-2-marker.value': 'l2', 
    'main-filter-3-marker.value': 'l2', 
    'main-graph-option-marker.value': 'ref', 
    'main-color.value': 'aggrnyl', 
    'main-highlight-save.n_clicks': None, 
    'main-highlight-toggle.value': 'off'
}
prefix = 'main'
suffix = 'marker.value'

In [ ]:
a = MSpec.objects.filter(observation__seq_id__iexact='mcam00128')

In [ ]:
modeldict(a[0])

In [ ]:
%%time
queryset = MSpec.objects.all()
filter_df = filter_df_from_queryset(queryset)


In [ ]:
%%time
queryset = truncate_queryset_for_missing_properties(
    settings,
    queryset,
    prefix,
    suffix,
    filter_df
)
queryset = truncate_queryset_for_missing_properties(
    settings,
    queryset,
    prefix,
    suffix,
    filter_df
)
queryset = truncate_queryset_for_missing_properties(
    settings,
    queryset,
    prefix,
    suffix,
    filter_df
)

In [ ]:
%lter_df['l1']

In [ ]:
%%time
m.filters

In [ ]:
%%time
a = []
for m in MSpec.objects.all():
    m.filter_values(None, False)

In [ ]:
%debug

In [ ]:
2000 * 500

In [ ]:
len(MSpec.objects.all())

In [ ]:
modeldict(m)

In [ ]:
spectrum.filters

In [ ]:
%debug

In [ ]:
m = pd.read_csv(input_fs.getsyspath(spec_file))

In [ ]:
m['R4'].astype(float)

In [ ]:
spec_file

In [ ]:
metadata

In [ ]:
modeldict(
    MSpec.objects.all()[0]
)

In [ ]:
seq_id

In [ ]:
frame

In [ ]:
obs = MSpec.objects.filter(observation__seq_id__iexact='mcam06328')

In [ ]:
obs

In [ ]:
from time import time

In [ ]:
t = time()
time() - t

In [ ]:
m = obs[0]

In [ ]:
m.as_table()

In [ ]:
modeldict(
    obs[0]
)

In [2]:
all_named_css = "aliceblue, antiquewhite, aqua, aquamarine, azure, beige, bisque, black, blanchedalmond, blue, blueviolet, brown, burlywood, cadetblue, chartreuse, chocolate, coral, cornflowerblue, cornsilk, crimson, cyan, darkblue, darkcyan, darkgoldenrod, darkgray, darkgrey, darkgreen, darkkhaki, darkmagenta, darkolivegreen, darkorange, darkorchid, darkred, darksalmon, darkseagreen, darkslateblue, darkslategray, darkslategrey, darkturquoise, darkviolet, deeppink, deepskyblue, dimgray, dimgrey, dodgerblue, firebrick, floralwhite, forestgreen, fuchsia, gainsboro, ghostwhite, gold, goldenrod, gray, grey, green, greenyellow, honeydew, hotpink, indianred, indigo, ivory, khaki, lavender, lavenderblush, lawngreen, lemonchiffon, lightblue, lightcoral, lightcyan, lightgoldenrodyellow, lightgray, lightgrey, lightgreen, lightpink, lightsalmon, lightseagreen, lightskyblue, lightslategray, lightslategrey, lightsteelblue, lightyellow, lime, limegreen, linen, magenta, maroon, mediumaquamarine, mediumblue, mediumorchid, mediumpurple, mediumseagreen, mediumslateblue, mediumspringgreen, mediumturquoise, mediumvioletred, midnightblue, mintcream, mistyrose, moccasin, navajowhite, navy, oldlace, olive, olivedrab, orange, orangered, orchid, palegoldenrod, palegreen, paleturquoise, palevioletred, papayawhip, peachpuff, peru, pink, plum, powderblue, purple, red, rosybrown, royalblue, rebeccapurple, saddlebrown, salmon, sandybrown, seagreen, seashell, sienna, silver, skyblue, slateblue, slategray, slategrey, snow, springgreen, steelblue, tan, teal, thistle, tomato, turquoise, violet, wheat, white, whitesmoke, yellow, yellowgreen"

In [9]:
for color in all_named_css.split(", "):
    print("{\"label\": \"" + color + "\", \"value\": \"" + color + "\"},")

{"label": "aliceblue", "value": "aliceblue"},
{"label": "antiquewhite", "value": "antiquewhite"},
{"label": "aqua", "value": "aqua"},
{"label": "aquamarine", "value": "aquamarine"},
{"label": "azure", "value": "azure"},
{"label": "beige", "value": "beige"},
{"label": "bisque", "value": "bisque"},
{"label": "black", "value": "black"},
{"label": "blanchedalmond", "value": "blanchedalmond"},
{"label": "blue", "value": "blue"},
{"label": "blueviolet", "value": "blueviolet"},
{"label": "brown", "value": "brown"},
{"label": "burlywood", "value": "burlywood"},
{"label": "cadetblue", "value": "cadetblue"},
{"label": "chartreuse", "value": "chartreuse"},
{"label": "chocolate", "value": "chocolate"},
{"label": "coral", "value": "coral"},
{"label": "cornflowerblue", "value": "cornflowerblue"},
{"label": "cornsilk", "value": "cornsilk"},
{"label": "crimson", "value": "crimson"},
{"label": "cyan", "value": "cyan"},
{"label": "darkblue", "value": "darkblue"},
{"label": "darkcyan", "value": "darkcyan

In [11]:
all_marker_types = [0, '0', 'circle', 100, '100', 'circle-open', 200, '200', 'circle-dot', 300, '300', 'circle-open-dot', 1, '1', 'square', 101, '101', 'square-open', 201, '201', 'square-dot', 301, '301', 'square-open-dot', 2, '2', 'diamond', 102, '102', 'diamond-open', 202, '202', 'diamond-dot', 302, '302', 'diamond-open-dot', 3, '3', 'cross', 103, '103', 'cross-open', 203, '203', 'cross-dot', 303, '303', 'cross-open-dot', 4, '4', 'x', 104, '104', 'x-open', 204, '204', 'x-dot', 304, '304', 'x-open-dot', 5, '5', 'triangle-up', 105, '105', 'triangle-up-open', 205, '205', 'triangle-up-dot', 305, '305', 'triangle-up-open-dot', 6, '6', 'triangle-down', 106, '106', 'triangle-down-open', 206, '206', 'triangle-down-dot', 306, '306', 'triangle-down-open-dot', 7, '7', 'triangle-left', 107, '107', 'triangle-left-open', 207, '207', 'triangle-left-dot', 307, '307', 'triangle-left-open-dot', 8, '8', 'triangle-right', 108, '108', 'triangle-right-open', 208, '208', 'triangle-right-dot', 308, '308', 'triangle-right-open-dot', 9, '9', 'triangle-ne', 109, '109', 'triangle-ne-open', 209, '209', 'triangle-ne-dot', 309, '309', 'triangle-ne-open-dot', 10, '10', 'triangle-se', 110, '110', 'triangle-se-open', 210, '210', 'triangle-se-dot', 310, '310', 'triangle-se-open-dot', 11, '11', 'triangle-sw', 111, '111', 'triangle-sw-open', 211, '211', 'triangle-sw-dot', 311, '311', 'triangle-sw-open-dot', 12, '12', 'triangle-nw', 112, '112', 'triangle-nw-open', 212, '212', 'triangle-nw-dot', 312, '312', 'triangle-nw-open-dot', 13, '13', 'pentagon', 113, '113', 'pentagon-open', 213, '213', 'pentagon-dot', 313, '313', 'pentagon-open-dot', 14, '14', 'hexagon', 114, '114', 'hexagon-open', 214, '214', 'hexagon-dot', 314, '314', 'hexagon-open-dot', 15, '15', 'hexagon2', 115, '115', 'hexagon2-open', 215, '215', 'hexagon2-dot', 315, '315', 'hexagon2-open-dot', 16, '16', 'octagon', 116, '116', 'octagon-open', 216, '216', 'octagon-dot', 316, '316', 'octagon-open-dot', 17, '17', 'star', 117, '117', 'star-open', 217, '217', 'star-dot', 317, '317', 'star-open-dot', 18, '18', 'hexagram', 118, '118', 'hexagram-open', 218, '218', 'hexagram-dot', 318, '318', 'hexagram-open-dot', 19, '19', 'star-triangle-up', 119, '119', 'star-triangle-up-open', 219, '219', 'star-triangle-up-dot', 319, '319', 'star-triangle-up-open-dot', 20, '20', 'star-triangle-down', 120, '120', 'star-triangle-down-open', 220, '220', 'star-triangle-down-dot', 320, '320', 'star-triangle-down-open-dot', 21, '21', 'star-square', 121, '121', 'star-square-open', 221, '221', 'star-square-dot', 321, '321', 'star-square-open-dot', 22, '22', 'star-diamond', 122, '122', 'star-diamond-open', 222, '222', 'star-diamond-dot', 322, '322', 'star-diamond-open-dot', 23, '23', 'diamond-tall', 123, '123', 'diamond-tall-open', 223, '223', 'diamond-tall-dot', 323, '323', 'diamond-tall-open-dot', 24, '24', 'diamond-wide', 124, '124', 'diamond-wide-open', 224, '224', 'diamond-wide-dot', 324, '324', 'diamond-wide-open-dot', 25, '25', 'hourglass', 125, '125', 'hourglass-open', 26, '26', 'bowtie', 126, '126', 'bowtie-open', 27, '27', 'circle-cross', 127, '127', 'circle-cross-open', 28, '28', 'circle-x', 128, '128', 'circle-x-open', 29, '29', 'square-cross', 129, '129', 'square-cross-open', 30, '30', 'square-x', 130, '130', 'square-x-open', 31, '31', 'diamond-cross', 131, '131', 'diamond-cross-open', 32, '32', 'diamond-x', 132, '132', 'diamond-x-open', 33, '33', 'cross-thin', 133, '133', 'cross-thin-open', 34, '34', 'x-thin', 134, '134', 'x-thin-open', 35, '35', 'asterisk', 135, '135', 'asterisk-open', 36, '36', 'hash', 136, '136', 'hash-open', 236, '236', 'hash-dot', 336, '336', 'hash-open-dot', 37, '37', 'y-up', 137, '137', 'y-up-open', 38, '38', 'y-down', 138, '138', 'y-down-open', 39, '39', 'y-left', 139, '139', 'y-left-open', 40, '40', 'y-right', 140, '140', 'y-right-open', 41, '41', 'line-ew', 141, '141', 'line-ew-open', 42, '42', 'line-ns', 142, '142', 'line-ns-open', 43, '43', 'line-ne', 143, '143', 'line-ne-open', 44, '44', 'line-nw', 144, '144', 'line-nw-open', 45, '45', 'arrow-up', 145, '145', 'arrow-up-open', 46, '46', 'arrow-down', 146, '146', 'arrow-down-open', 47, '47', 'arrow-left', 147, '147', 'arrow-left-open', 48, '48', 'arrow-right', 148, '148', 'arrow-right-open', 49, '49', 'arrow-bar-up', 149, '149', 'arrow-bar-up-open', 50, '50', 'arrow-bar-down', 150, '150', 'arrow-bar-down-open', 51, '51', 'arrow-bar-left', 151, '151', 'arrow-bar-left-open', 52, '52', 'arrow-bar-right', 152, '152', 'arrow-bar-right-open']
marker_str = []
for marker_type in all_marker_types:
    try: 
        int(marker_type)
    except ValueError:
        marker_str.append(
            {"label": "\"" + marker_type + "\"", "value": "\"" + marker_type + "\""}
        )
        

In [13]:
tuple(marker_str)

({'label': '"circle"', 'value': '"circle"'},
 {'label': '"circle-open"', 'value': '"circle-open"'},
 {'label': '"circle-dot"', 'value': '"circle-dot"'},
 {'label': '"circle-open-dot"', 'value': '"circle-open-dot"'},
 {'label': '"square"', 'value': '"square"'},
 {'label': '"square-open"', 'value': '"square-open"'},
 {'label': '"square-dot"', 'value': '"square-dot"'},
 {'label': '"square-open-dot"', 'value': '"square-open-dot"'},
 {'label': '"diamond"', 'value': '"diamond"'},
 {'label': '"diamond-open"', 'value': '"diamond-open"'},
 {'label': '"diamond-dot"', 'value': '"diamond-dot"'},
 {'label': '"diamond-open-dot"', 'value': '"diamond-open-dot"'},
 {'label': '"cross"', 'value': '"cross"'},
 {'label': '"cross-open"', 'value': '"cross-open"'},
 {'label': '"cross-dot"', 'value': '"cross-dot"'},
 {'label': '"cross-open-dot"', 'value': '"cross-open-dot"'},
 {'label': '"x"', 'value': '"x"'},
 {'label': '"x-open"', 'value': '"x-open"'},
 {'label': '"x-dot"', 'value': '"x-dot"'},
 {'label': '"